# Data Extractor: Epic Moment

This notebook extracts and anonymizes chat data for the dsc project.

It reads the data and anomyzes them and stores them in a new folder.







<a id='content'></a>
## High-Level-Contents of this Notebook

[Step 1: Find matches from 2 players and export data](#exportmatchdata)  
 - Selects from database (and dump into encrypted pickles)

[Step 2: Read Chat from matches of whole gaming session](#exportchats)  
 - Read chat data of matches
 - anonymize
 - dump into pickles

## Imports and required libraries

In [1]:
import os
import sys
import pickle
import dotenv
import pyAesCrypt

import numpy as np
import pandas as pd

from pathlib import Path
from datetime import datetime, timedelta
from pymongo import MongoClient



# Global pandas options
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)


### Utility Functions for writing data or reading data from the *prepared_data* folder

In [2]:
def get_password():
    basedir = get_dsc_data_file_path().parent
    dotenv.load_dotenv(os.path.join(basedir, '.env'))
    return os.environ.get('PICKLE_PW')

def get_dsc_data_file_path():
    src_path = Path(os.getcwd())
    parent_path = src_path.parent.absolute()
    dsc_data_path = parent_path.joinpath('dsc_data')

    if not os.path.exists(dsc_data_path):
        os.makedirs(dsc_data_path)
    return dsc_data_path

def get_player_data_file_path(playername):
    dsc_path = get_dsc_data_file_path()

    player_path = dsc_path.joinpath(playername)

    if not os.path.exists(player_path):
        os.makedirs(player_path)
    return player_path

def get_file_path(playername, file):
    player_path = get_player_data_file_path(playername)
    filename = f'{file}.pkl'
    aes_filename = f'{filename}.aes'
    return player_path.joinpath(filename), player_path.joinpath(aes_filename)

def read_prepared_file(playername, file):
    destination_file, aes_file = get_file_path(playername, file)
    pyAesCrypt.decryptFile(aes_file, destination_file, get_password())
    with open(destination_file, "rb") as src:
        obj = pickle.load(src)
        print(f'read {destination_file}')
    os.remove(destination_file)
    return obj

def read_prepared_df_file(playername, df_file, index_col):
    destination_file, aes_file = get_file_path(playername, df_file)
    pyAesCrypt.decryptFile(aes_file, destination_file, get_password())
    df = pd.read_csv(destination_file,index_col = index_col)
    print(f'read {destination_file}')
    os.remove(destination_file)
    return df

def write_prepared_file(obj, playername, file):
    destination_file, aes_file = get_file_path(playername, file)
    with open(destination_file, "wb") as dst:
        pickle.dump(obj, dst)
        print(f'wrote {destination_file}')
    pyAesCrypt.encryptFile(destination_file, aes_file, get_password())
    os.remove(destination_file)

def write_prepared_df_file(df, playername, file):
    destination_file, aes_file = get_file_path(playername, file)
    df.to_csv(destination_file)
    print(f'wrote {destination_file}')
    pyAesCrypt.encryptFile(destination_file, aes_file, get_password())
    os.remove(destination_file)

def delete_dsc_data():
    dsc_path = get_dsc_data_file_path()
    for d in os.listdir(dsc_path):
        folder_path = dsc_path.joinpath(d)
        if os.path.isdir(folder_path):
            for f in os.listdir(folder_path):
                file_path = folder_path.joinpath(f)
                os.remove(file_path)
            os.rmdir(folder_path)
        else:
            os.remove(folder_path)
    os.rmdir(dsc_path)

In [3]:
delete_dsc_data()

<a id='exportmatchdata'></a>
## Step 1: Find matches from 2 players and export data | ([Jump to Contents](#content))

### Establish Connection to local DB

This is only required for running the notebook with database access.

In [4]:

#Creating a pymongo client
client = MongoClient('localhost', 27017)

#Getting the database instance
db = client['epicml']

### Export Matches of Streamers: tobiasfate and noway4u_sir

noway4u_sir is a streamer from Germany and his chat language is usually in German.

Tobiasfate is a streamer from Canada and his chat language is usually English.

In [5]:
def write_50_lol_matches(username):
    mappings = list(db['twitch_channel_summoner_mapping'].find({'twitch_channel': username, 'status': 'COMPLETED'}))
    pks = [channel['summoner_id'] for channel in mappings]
    summoners = list(db['lol_summoner'].find({'_id': {'$in': pks}}))
    matches = list(db['lol_match_participant_summary'].find({"puuid": {'$in': [summoner['puuid'] for summoner in summoners]}}))
    match_ids = [match['matchId'] for match in matches]
    match_ids = match_ids[:50]

    match_summaries = list(db['lol_match_summary'].find({"matchId": {'$in': match_ids}}))
    match_participant_summaries = list(db['lol_match_participant_summary'].find({"matchId": {'$in': match_ids}}))
    match_timelines = list(db['lol_match_timeline'].find({"matchId": {'$in': match_ids}}))

    write_prepared_file(match_summaries, username, "match_summaries")

    # resolve 1:n problem
    for match_id in match_ids:
        match_participants = []
        for participant in match_participant_summaries:
            if participant['matchId'] == match_id:
                match_participants.append(participant)
        filename = f"match_participant_summaries_{match_id}"
        write_prepared_file(match_participants, username, filename)

    for timeline in match_timelines:
        match_id = timeline['matchId']
        write_prepared_file(timeline, username, f"match_timeline_{match_id}")

    return match_participant_summaries, match_summaries, match_timelines

In [6]:
tf_matches, tf_summaries, tf_timelines = write_50_lol_matches('tobiasfate')

wrote /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-final-project-epic-ml-visualizer/dsc_data/tobiasfate/match_summaries.pkl
wrote /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-final-project-epic-ml-visualizer/dsc_data/tobiasfate/match_participant_summaries_NA1_4047445107.pkl
wrote /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-final-project-epic-ml-visualizer/dsc_data/tobiasfate/match_participant_summaries_NA1_4034134889.pkl
wrote /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-final-project-epic-ml-visualizer/dsc_data/tobiasfate/match_participant_summaries_NA1_4033596036.pkl
wrote /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-final-project-epic-ml-visualizer/dsc_data/tobiasfate/match_participant_summaries_NA1_4033488515.pkl
wrote /Users/doba/Documents/TechnikumWien/3. Semester/Sol

wrote /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-final-project-epic-ml-visualizer/dsc_data/tobiasfate/match_participant_summaries_NA1_4081298888.pkl
wrote /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-final-project-epic-ml-visualizer/dsc_data/tobiasfate/match_participant_summaries_NA1_4080918526.pkl
wrote /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-final-project-epic-ml-visualizer/dsc_data/tobiasfate/match_participant_summaries_NA1_4080885402.pkl
wrote /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-final-project-epic-ml-visualizer/dsc_data/tobiasfate/match_participant_summaries_NA1_4080848287.pkl
wrote /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-final-project-epic-ml-visualizer/dsc_data/tobiasfate/match_participant_summaries_NA1_4080660913.pkl
wrote /Users/doba/Documents/Te

wrote /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-final-project-epic-ml-visualizer/dsc_data/tobiasfate/match_timeline_NA1_4166666165.pkl
wrote /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-final-project-epic-ml-visualizer/dsc_data/tobiasfate/match_timeline_NA1_4166679015.pkl
wrote /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-final-project-epic-ml-visualizer/dsc_data/tobiasfate/match_timeline_NA1_4166754410.pkl
wrote /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-final-project-epic-ml-visualizer/dsc_data/tobiasfate/match_timeline_NA1_4166758441.pkl
wrote /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-final-project-epic-ml-visualizer/dsc_data/tobiasfate/match_timeline_NA1_4166761418.pkl
wrote /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-f

In [7]:
nw_matches, nw_summaries, nw_timelines = write_50_lol_matches('noway4u_sir')

wrote /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-final-project-epic-ml-visualizer/dsc_data/noway4u_sir/match_summaries.pkl
wrote /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-final-project-epic-ml-visualizer/dsc_data/noway4u_sir/match_participant_summaries_NA1_4112918754.pkl
wrote /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-final-project-epic-ml-visualizer/dsc_data/noway4u_sir/match_participant_summaries_NA1_4113114496.pkl
wrote /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-final-project-epic-ml-visualizer/dsc_data/noway4u_sir/match_participant_summaries_NA1_4113054688.pkl
wrote /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-final-project-epic-ml-visualizer/dsc_data/noway4u_sir/match_participant_summaries_NA1_4113031259.pkl
wrote /Users/doba/Documents/TechnikumWien/3. Semeste

wrote /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-final-project-epic-ml-visualizer/dsc_data/noway4u_sir/match_participant_summaries_NA1_4122872258.pkl
wrote /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-final-project-epic-ml-visualizer/dsc_data/noway4u_sir/match_participant_summaries_NA1_4122840220.pkl
wrote /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-final-project-epic-ml-visualizer/dsc_data/noway4u_sir/match_participant_summaries_NA1_4122101140.pkl
wrote /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-final-project-epic-ml-visualizer/dsc_data/noway4u_sir/match_participant_summaries_NA1_4121762938.pkl
wrote /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-final-project-epic-ml-visualizer/dsc_data/noway4u_sir/match_participant_summaries_NA1_4120609128.pkl
wrote /Users/doba/Documen

wrote /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-final-project-epic-ml-visualizer/dsc_data/noway4u_sir/match_timeline_NA1_4121616007.pkl
wrote /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-final-project-epic-ml-visualizer/dsc_data/noway4u_sir/match_timeline_NA1_4121762938.pkl
wrote /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-final-project-epic-ml-visualizer/dsc_data/noway4u_sir/match_timeline_NA1_4121786986.pkl
wrote /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-final-project-epic-ml-visualizer/dsc_data/noway4u_sir/match_timeline_NA1_4121902442.pkl
wrote /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-final-project-epic-ml-visualizer/dsc_data/noway4u_sir/match_timeline_NA1_4121986304.pkl
wrote /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/

<a id='findchats'></a>
## Step 2: Read Chat from matches of whole gaming session | ([Jump to Contents](#content))

In [ ]:
# helper functions for reading chat data

def map_chatlog_row(text: str):
    try:
        splitted = text.split(': ', 1)
        if len(splitted) == 1:
            return None

        if len(splitted) != 2:
            index_date_end = text.index(']')
            time = text[1: index_date_end]
            time = time.replace(' ', 'T')
            channel_start_idx = text.index(' ', index_date_end + 1)
            channel_end_idx = text.index(' ', channel_start_idx + 1)
            channel = text[channel_start_idx + 1:channel_end_idx]
            channel = channel[1:]
            text = text[channel_end_idx:]
            text = text.strip()
            return {'datetime': time, 'channel': channel, 'text': text}
        else:
            left = splitted[0]
            right = splitted[1]

            lefties = left.split(" ")
            if len(lefties) == 4:
                time = f'{lefties[0][1:]}T{lefties[1][:-1]}'
                channel = lefties[2][1:]
                username = lefties[3]
            else:
                time = lefties[0][1:-1]
                channel = lefties[1][1:]
                username = lefties[2]

            text = right.strip()

            return {'datetime': time, 'channel': channel, 'author_name':username, 'text': text}
    except Exception as e:
        _logger.error(f"invalid format of line '{text}")
        raise e


def ts_from_str(dt_str) -> int:
    dt_str = dt_str.replace(' ', 'T')
    if '.' not in dt_str:
        dt_str = dt_str + ".0"
    dt = strptime(dt_str, '%Y-%m-%dT%H:%M:%S.%f')[0:6]
    return int(datetime(*dt).replace(tzinfo=timezone.utc).timestamp())

def utc_datetime_from_ts(ts) -> datetime:
    return datetime.utcfromtimestamp(ts)

In [9]:
def get_channel_path(playername):
    rawdata_path = Path('/Users/doba/Documents/epicml_data/dataservice_instance/rawdata')
    return rawdata_path.joinpath(playername)

def get_logfiles_by_pattern(channel_path, log_patterns):
    return [channel_path.joinpath(file) for pattern in log_patterns for file in os.listdir(channel_path) if pattern in file]


ModuleNotFoundError: No module named 'requests'

In [ ]:
## Functions to read raw log files for matches
def convert_match_start_end_summaries(match_summaries):
    match_ids = []
    durations = []
    start_dates = []
    end_dates = []
    for match in match_summaries:
        match_ids.append(match['matchId'])
        durations.append(match['gameDuration_ms'])
        start_dates.append(match['gameStartTimestamp_date'])
        if match['gameDuration_ms'] < 10000:
            end_dates.append(match['gameStartTimestamp_date'] + timedelta(seconds=match['gameDuration_ms']))
        else:
            end_dates.append(match['gameStartTimestamp_date'] + timedelta(milliseconds=match['gameDuration_ms']))
    df_matches = pd.DataFrame({
        'matchId': match_ids,
        'duration_ms': durations,
        'start_date': start_dates,
        'end_date': end_dates
    })
    df_matches.set_index('start_date', inplace=True)
    df_matches.sort_index(inplace=True)
    df_matches['start_log_file_pattern'] = df_matches.index.map(lambda x: x.strftime("%y_%m_%d.log"))
    df_matches['end_log_file_pattern'] = df_matches.loc[:, 'end_date'].map(lambda x: x.strftime("%y_%m_%d.log"))
    return df_matches

tf_matches_dt_summ = convert_match_start_end_summaries(tf_summaries)
nw_matches_dt_summ = convert_match_start_end_summaries(nw_summaries)

In [ ]:
tf_matches_dt_summ

In [ ]:
from shared.common.analyzer.twitch_logs import possible_botlist
possible_botlist

In [ ]:
def nearest(items, pivot):
    time_diff = np.abs(items - pivot)
    return time_diff.argmin(0)

def read_chat_of_matches(playername, matches_dt_summ):
    start_patterns = matches_dt_summ['start_log_file_pattern']
    end_patterns = matches_dt_summ['end_log_file_pattern']
    log_patterns = pd.concat([start_patterns, end_patterns]).unique()

    channel_path = get_channel_path(playername)

    raw_messages = []
    for file in get_logfiles_by_pattern(channel_path, log_patterns):
        with open(file, 'r') as log_file:
            for line in log_file:
                if len(line) > 0:
                    chat_row = map_chatlog_row(line)
                    if chat_row is None:
                        continue
                    raw_messages.append(chat_row)

    df_raw_messages = pd.DataFrame(raw_messages)
    df_raw_messages['datetime'] = pd.to_datetime(df_raw_messages['datetime'])
    df_raw_messages['channel'] = df_raw_messages['channel'].str.lower()
    df_raw_messages['author_name'] = df_raw_messages['author_name'].str.lower()
    df_raw_messages['text'] = df_raw_messages['text'].str.lower()
    df_raw_messages.set_index('datetime', inplace=True)


    df_raw_messages.drop(['author_id'], axis=1, inplace=True, errors = 'ignore')



    df_chat_messages = df_raw_messages.copy()
    df_chat_messages['matchId'] = None
    df_chat_messages['timecategory'] = 'NO_MATCH'

    df_col_locs = [df_chat_messages.columns.get_loc("matchId"), df_chat_messages.columns.get_loc("timecategory")]

    for index, row in matches_dt_summ.iterrows():
        start_ts = index.round('1s')
        end_ts = (row['end_date'] + timedelta(seconds=1)).round('1s')
        matchId = row['matchId']
        start_pre_match_ts = (start_ts - timedelta(seconds=60)).replace(microsecond=0)
        start_pre_match_iloc = nearest(df_chat_messages.index, start_pre_match_ts)
        start_match_iloc = nearest(df_chat_messages.index, start_ts)
        end_match_iloc = nearest(df_chat_messages.index, end_ts)
        end_post_match_ts = (end_ts + timedelta(seconds=60)).replace(microsecond=0)
        end_post_match_iloc = nearest(df_chat_messages.index, end_post_match_ts)


        df_chat_messages.iloc[start_pre_match_iloc:start_match_iloc, df_col_locs] = [matchId, 'BEFORE_MATCH']

        df_chat_messages.iloc[start_match_iloc:end_match_iloc, df_col_locs] = [matchId, 'DURING_MATCH']

        df_chat_messages.iloc[end_match_iloc:end_post_match_iloc, df_col_locs] = [matchId, 'AFTER_MATCH']

    # Add feature is_chatbot
    bots_lowercase = [bot.lower() for bot in possible_botlist]
    df_chat_messages['chatbot'] = df_chat_messages['author_name'].apply(lambda x: x.lower() in bots_lowercase)

    # Add feature Personal-Message
    df_chat_messages['personal_msg'] = df_chat_messages['text'].apply(lambda x: x.startswith('@') and not x.startswith(f'@{playername}') and ' ' in x)

    # Add Feature command
    df_chat_messages['command'] = df_chat_messages['text'].apply(lambda x: x.startswith('!'))

    # only return rows before/in or after a match
    return df_chat_messages[~df_chat_messages['matchId'].isna()]

The 'ugly' join with nearest is needed, because both dataframes do not have an index with a fixed frequency. So there are leaks in the index of both dataframes.

In [ ]:
tf_chat_df = read_chat_of_matches('tobiasfate', tf_matches_dt_summ)
nw_chat_df = read_chat_of_matches('noway4u_sir', nw_matches_dt_summ)

In [ ]:
nw_chat_df

In [80]:
write_prepared_df_file(tf_chat_df, 'tobiasfate', 'chat_df')
write_prepared_df_file(nw_chat_df, 'noway4u_sir', 'chat_df')

wrote /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-final-project-epic-ml-visualizer/dsc_data/tobiasfate/chat_df.pkl
wrote /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-final-project-epic-ml-visualizer/dsc_data/noway4u_sir/chat_df.pkl


In [ ]:
read_df = read_prepared_df_file('noway4u_sir', 'chat_df', 'datetime')
read_df

In [ ]:
Es gibt einige Matches, wo es keinen Chat gibt!

In [ ]:
# Export ends here!

## Read from dsc_file folder

In [74]:
def get_match_history_of_player(player_name):
    match_summaries = read_prepared_file(player_name, "match_summaries")
    return pd.DataFrame(match_summaries)

history = get_match_history_of_player("noway4u_sir")
history

read /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-final-project-epic-ml-visualizer/dsc_data/noway4u_sir/match_summaries.pkl


,_id,gameCreation,gameEndTimestamp,gameMode,gameName,gameStartTimestamp,gameType,gameVersion,mapId,teams,tournamentCode,dataVersion,matchId,participants,gameDuration_ms,req_server,gameCreation_date,gameStartTimestamp_date,matchTimelineId
0,619cde1976e6c35572ba3bd5,1637599884000,1637602003183,CLASSIC,teambuilder-match-4112918754,1637599911179,MATCHED_GAME,11.23.409.111,11,"[{'bans': [{'championId': 164, 'pickTurn': 1},...",,2,NA1_4112918754,[-L314LjYwt1J1K_suPbBHEwpQCXJlmxnEl8cszKarzYgm...,2091,NA1,2021-11-22 16:51:24,2021-11-22 16:51:51.179,61b323f63deba7f22f137b00
1,619d028676e6c35572bb9f5f,1637608331000,1637610122626,CLASSIC,teambuilder-match-4113031259,1637608380115,MATCHED_GAME,11.23.409.111,11,"[{'bans': [{'championId': 711, 'pickTurn': 1},...",,2,NA1_4113031259,[o2l4tKoPPlpq03-iXlIF8qXSiOEjWQwCuSpNfTexpjzIj...,1742,NA1,2021-11-22 19:12:11,2021-11-22 19:13:00.115,61b326f23deba7f22f138a26
2,619d028576e6c35572bb9f49,1637610860000,1637612765278,CLASSIC,teambuilder-match-4113054688,1637610899744,MATCHED_GAME,11.23.409.111,11,"[{'bans': [{'championId': 56, 'pickTurn': 1}, ...",,2,NA1_4113054688,[WpYAA8AZzkj7b9iaeVSc097kNxuosLi5TXHWBXu_78MYU...,1865,NA1,2021-11-22 19:54:20,2021-11-22 19:54:59.744,61b326f13deba7f22f138a1a
3,619ce17c76e6c35572ba6754,1637616155000,1637617674587,CLASSIC,teambuilder-match-4113114496,1637616179168,MATCHED_GAME,11.23.409.111,11,"[{'bans': [{'championId': 119, 'pickTurn': 1},...",,2,NA1_4113114496,[9EQgNUWVj68SICPqUM0TqW4IaGg-Of9jJf0by3DxTmMUN...,1495,NA1,2021-11-22 21:22:35,2021-11-22 21:22:59.168,61b323ff3deba7f22f137b08
4,619e036e76e6c35572bdbef1,1637684058000,1637685982726,CLASSIC,teambuilder-match-4114353407,1637684127933,MATCHED_GAME,11.23.409.111,11,"[{'bans': [{'championId': 45, 'pickTurn': 1}, ...",,2,NA1_4114353407,[Ja68D8TIcIVF9Ny2WRbj_LnRPr4nJFfSIx82wuJCZVm6d...,1854,NA1,2021-11-23 16:14:18,2021-11-23 16:15:27.933,61b32e093deba7f22f13b59d
5,619e8b6f76e6c35572c19126,1637692152000,1637694244814,CLASSIC,teambuilder-match-4114384279,1637692271162,MATCHED_GAME,11.23.409.111,11,"[{'bans': [{'championId': 26, 'pickTurn': 1}, ...",,2,NA1_4114384279,[nUWW7AAdG6Pm6QudRPxaX9pyqwPjsm9VMwGNZIH2T6anT...,1973,NA1,2021-11-23 18:29:12,2021-11-23 18:31:11.162,61b335a73deba7f22f13e0d0
6,619e31a476e6c35572bfb565,1637689741000,1637691469132,CLASSIC,teambuilder-match-4114410624,1637689801794,MATCHED_GAME,11.23.409.111,11,"[{'bans': [{'championId': 102, 'pickTurn': 1},...",,2,NA1_4114410624,[p2M7IkGCwsRPqd7I8VYGi8vcoWtY_W5YwOfhEViTW9j-Z...,1667,NA1,2021-11-23 17:49:01,2021-11-23 17:50:01.794,61b32fec3deba7f22f13c1b0
7,619f7ac476e6c35572c4497a,1637707766000,1637710434343,CLASSIC,teambuilder-match-4114572094,1637707790439,MATCHED_GAME,11.23.409.111,11,"[{'bans': [{'championId': 38, 'pickTurn': 1}, ...",,2,NA1_4114572094,[zI4UzklmqgK68E09CFgMJJFk9QEQ0i0uw18nIN2smAc4f...,2643,NA1,2021-11-23 22:49:26,2021-11-23 22:49:50.439,61b338143deba7f22f13ea73
8,619e5b5d76e6c35572c10ef2,1637724795000,1637726251491,CLASSIC,teambuilder-match-4114896489,1637724852190,MATCHED_GAME,11.23.409.111,11,"[{'bans': [{'championId': 147, 'pickTurn': 1},...",,2,NA1_4114896489,[2wsaXabYW_1Z6iXzGg42WqrUtAuu4vnqgReStfiUm5rUe...,1399,NA1,2021-11-24 03:33:15,2021-11-24 03:34:12.190,61b331c93deba7f22f13c9fb
9,619f7bb276e6c35572c45926,1637780082000,1637781037109,CLASSIC,teambuilder-match-4115483709,1637780108900,MATCHED_GAME,11.23.409.111,11,"[{'bans': [{'championId': 26, 'pickTurn': 1}, ...",,2,NA1_4115483709,[EvlGs3-gj2E2KihdjvqqhH1axtEm2YQeeBY57dRVeLcyw...,928,NA1,2021-11-24 18:54:42,2021-11-24 18:55:08.900,61b338d73deba7f22f13ed6d


In [75]:
history['matchId']

0     NA1_4112918754
1     NA1_4113031259
2     NA1_4113054688
3     NA1_4113114496
4     NA1_4114353407
5     NA1_4114384279
6     NA1_4114410624
7     NA1_4114572094
8     NA1_4114896489
9     NA1_4115483709
10    NA1_4115495602
11    NA1_4115509715
12    NA1_4116366488
13    NA1_4116741548
14    NA1_4116748183
15    NA1_4117096306
16    NA1_4117881867
17    NA1_4117897084
18    NA1_4117971821
19    NA1_4117985954
20    NA1_4118140558
21    NA1_4118145454
22    NA1_4119331516
23    NA1_4119412825
24    NA1_4119487449
25    NA1_4119574152
26    NA1_4119588134
27    NA1_4119673297
28    NA1_4119907297
29    NA1_4119985066
30    NA1_4120457571
31    NA1_4120580479
32    NA1_4120609128
33    NA1_4120666300
34    NA1_4120734190
35    NA1_4121510202
36    NA1_4121512011
37    NA1_4121616007
38    NA1_4121762938
39    NA1_4121786986
40    NA1_4121902442
41    NA1_4121986304
42    NA1_4122101140
43    NA1_4122840220
44    NA1_4122855882
45    NA1_4122872258
46    NA1_4123006893
47    NA1_412

## Read match from player

In [77]:
def get_match_summary_array(player_name, match_id):
    filename = f"match_participant_summaries_{match_id}"
    return read_prepared_file(player_name, filename)

match_summary = get_match_summary_array("noway4u_sir", "NA1_4112918754")

match_summary[0]

read /Users/doba/Documents/TechnikumWien/3. Semester/Solution Deployment & Communication/dsc-final-project-epic-ml-visualizer/dsc_data/noway4u_sir/match_participant_summaries_NA1_4112918754.pkl


{'_id': ObjectId('619cde1976e6c35572ba3bd6'),
 'assists': 9,
 'baronKills': 0,
 'bountyLevel': 0,
 'champExperience': 15883,
 'champLevel': 16,
 'championId': 36,
 'championName': 'DrMundo',
 'championTransform': 0,
 'consumablesPurchased': 1,
 'damageDealtToBuildings': 2123,
 'damageDealtToObjectives': 4279,
 'damageDealtToTurrets': 2123,
 'damageSelfMitigated': 68639,
 'deaths': 10,
 'detectorWardsPlaced': 0,
 'doubleKills': 0,
 'dragonKills': 0,
 'firstBloodAssist': True,
 'firstBloodKill': False,
 'firstTowerAssist': False,
 'firstTowerKill': False,
 'gameEndedInEarlySurrender': False,
 'gameEndedInSurrender': False,
 'goldEarned': 11249,
 'goldSpent': 11150,
 'individualPosition': 'TOP',
 'inhibitorKills': 0,
 'inhibitorTakedowns': 0,
 'inhibitorsLost': 1,
 'item0': 3082,
 'item1': 3065,
 'item2': 6662,
 'item3': 3047,
 'item4': 3075,
 'item5': 1029,
 'item6': 3340,
 'itemsPurchased': 23,
 'killingSprees': 1,
 'kills': 2,
 'lane': 'BOTTOM',
 'largestCriticalStrike': 17,
 'largestK

In [78]:
[match['summonerName'] for match in match_summary]

['Hi im Marcos',
 'S E D Z A M A',
 'coinflipper420',
 'German Aimbot',
 'www JZVapes com',
 'FENG ZHUA',
 'Most Deaths',
 'pucciness',
 'MinGuo',
 'Gloomy Shenteemo']

In [85]:
## Chat of a single match
read_df[read_df['matchId'] == 'NA1_4112918754']['timecategory'].value_counts()

1609

### Transform to word list without direct repetitions


In [ ]:
from shared.common.util.textreducer import chatmessage_pipeline

In [ ]:
df_session_chat_messages['deduplicated'] = df_session_chat_messages['text'].apply(chatmessage_pipeline)
df_session_chat_messages

### Create word clouds with text by match type (pentakill (=epic), bad, good)

In [ ]:
words_pentakill_match = df_session_chat_messages[no_chatbot_comm_msg_mask & (df_session_chat_messages['matchId'] == match_id_pentakill)]['deduplicated'].str.cat(sep=' ')
words_lost_min_kills_match = df_session_chat_messages[no_chatbot_comm_msg_mask & (df_session_chat_messages['matchId'] == match_id_lost_min_kills)]['deduplicated'].str.cat(sep=' ')
words_win_max_killsprees_match = df_session_chat_messages[no_chatbot_comm_msg_mask & (df_session_chat_messages['matchId'] == match_id_win_max_killsprees)]['deduplicated'].str.cat(sep=' ')

In [ ]:
wordcloud_pentakill = WordCloud(width= 3000, height = 2000, random_state=1, colormap='Pastel1', collocations=False, stopwords = STOPWORDS).generate(words_pentakill_match)
wordcloud_lost_min_kills = WordCloud(width= 3000, height = 2000, random_state=1, colormap='Pastel1', collocations=False, stopwords = STOPWORDS).generate(words_lost_min_kills_match)
wordcloud_win_max_killsprees = WordCloud(width= 3000, height = 2000, random_state=1, colormap='Pastel1', collocations=False, stopwords = STOPWORDS).generate(words_win_max_killsprees_match)

In [ ]:
with sns.plotting_context('talk'):
    fig, ax = plt.subplots(figsize=(8, 16), nrows=3, ncols=1)
    plt.tight_layout()
    ax[0].set_title('Wordcloud Pentakill')
    ax[0].imshow(wordcloud_pentakill)
    ax[0].axis('off')

    ax[1].set_title('Wordcloud Lost-Min-Kills')
    ax[1].imshow(wordcloud_lost_min_kills)
    ax[1].axis('off')

    ax[2].set_title('Wordcloud Win-Max-Killsprees')
    ax[2].imshow(wordcloud_win_max_killsprees)
    ax[2].axis('off')
    plt.show()

Nicht rein Englisch, sondern eher Twitchish. Es gibt noise, der auf den Channel hinweist (loltyler1, tyler, ...) und einige schimpfwörter. Daher müssen chatbot Nachrichten und commands der viewer ausgenommen werden.

### Number of messages by chat user during matches from gaming session

In [ ]:
with sns.plotting_context('talk'):
    f, ax = plt.subplots(1, 2, figsize = (20, 30))
    f.tight_layout(pad=10.0)
    messasges_by_user = df_session_chat_messages[(df_session_chat_messages['timecategory'] == 'DURING_MATCH')].groupby(['author_name'])
        .size()
        .reset_index(name='count')
        .sort_values('count', ascending=False)

    quantiles = messasges_by_user['count'].quantile([0.25,0.5,0.75,0.99])
    num_messages_min = quantiles.index[-1]


    sns.barplot(ax=ax[0], y='author_name', x='count', data=messasges_by_user[messasges_by_user['count'] > quantiles.values[-1]])
    ax[0].set_xlabel('Anzahl der Nachrichten')
    ax[0].set_ylabel('')
    ax[0].set_title(f'Chat messages by all Top {round((1 - num_messages_min) * 100)} % users')

    messasges_by_user = df_session_chat_messages[no_chatbot_comm_msg_mask & (df_session_chat_messages['timecategory'] == 'DURING_MATCH')].groupby(['author_name'])
        .size()
        .reset_index(name='count')
        .sort_values('count', ascending=False)

    quantiles = messasges_by_user['count'].quantile([0.25,0.5,0.75,0.95, 0.99])

    num_messages_min = quantiles.index[-1]

    sns.barplot(ax=ax[1], y='author_name', x='count', data=messasges_by_user[messasges_by_user['count'] > quantiles.values[-1]])
    ax[1].set_xlabel('Anzahl der Nachrichten')
    ax[1].set_ylabel('')
    ax[1].set_title(f'Chat messages by Top {round((1 - num_messages_min) * 100)} % users without bot messages')


Die Chatbot Nachrichten (vor allem commands) beeinflussen die Counts schon, die Reihenfolgen der Top-Chatter kommen etwas durcheinander.

<a id='loadtimelines'></a>
## Step 3: Load Match Timelines from the previously selected Matches | ([Jump to Contents](#content))

In [ ]:
if notebook_mode == 'DEV':
    from riotwatcher import LolWatcher
    riot_api = LolWatcher('I nearly shared it ;)') # Personal key

    match_timeline_pentakill = riot_api.match.timeline_by_match(region='AMERICAS', match_id=match_id_pentakill)
    write_prepared_file(match_timeline_pentakill, 'match_timeline_pentakill')

    match_timeline_lost_min_kills = riot_api.match.timeline_by_match(region='AMERICAS', match_id=match_id_lost_min_kills)
    write_prepared_file(match_timeline_lost_min_kills, 'match_timeline_lost_min_kills')

    match_timeline_win_max_killsprees = riot_api.match.timeline_by_match(region='AMERICAS', match_id=match_id_win_max_killsprees)
    write_prepared_file(match_timeline_win_max_killsprees, 'match_timeline_win_max_killsprees')
else:
    match_timeline_pentakill = read_prepared_file('match_timeline_pentakill')
    match_timeline_lost_min_kills = read_prepared_file('match_timeline_lost_min_kills')
    match_timeline_win_max_killsprees = read_prepared_file('match_timeline_win_max_killsprees')


match_timeline_pentakill['matchId'] = match_id_pentakill
match_timeline_lost_min_kills['matchId'] = match_id_lost_min_kills
match_timeline_win_max_killsprees['matchId'] = match_id_win_max_killsprees


### Create Summoner-Of-Match-Map

In [ ]:
def create_match_summoner_dict(match_timeline):
    match_summoner_puuids = [p['puuid'] for p in match_timeline['info']['participants']]
    matchId = match_timeline['matchId']

    fname = f'summoners_of_match_{matchId}'
    if notebook_mode == 'DEV':
        summoners_of_match = list(db['lol_match_participant_summary'].find({
            '$and': [
                {"puuid": {'$in': match_summoner_puuids}},
                {"matchId": matchId}
            ]}, {'puuid': 1, 'summonerName': 1}))
        write_prepared_file(summoners_of_match, fname)
    else:
        summoners_of_match = read_prepared_file(fname)

    participant_map = {'0': {'puuid': '-', 'name': 'Minions'}}
    for p in match_timeline['info']['participants']:

        participant_map[str(p['participantId'])] = {
            'puuid': p['puuid'],
            'name': [summ['summonerName'] for summ in summoners_of_match if summ['puuid'] == p['puuid']][0]
        }
    return participant_map

In [ ]:
# Test Participant-Map
create_match_summoner_dict(match_timeline_pentakill)

In [ ]:
import shared.common.util.datetimeutil as datetimeutil

def calc_datetime(start_dt, delta_event):
    return start_dt + timedelta(milliseconds = delta_event)

def get_key_event_player(event_type):
    if event_type in ["ITEM_PURCHASED", "ITEM_DESTROYED", "ITEM_UNDO", "ITEM_SOLD", "SKILL_LEVEL_UP", "LEVEL_UP", "CHAMPION_TRANSFORM"]:
        return "participantId"
    elif event_type in ["CHAMPION_KILL", "CHAMPION_SPECIAL_KILL", "WARD_KILL", "ELITE_MONSTER_KILL", "BUILDING_KILL", "TURRET_PLATE_DESTROYED"]:
        return "killerId"
    elif event_type in ["WARD_PLACED"]:
        return "creatorId"
    elif event_type in ["PAUSE_END", "GAME_END"]:
        return None
    else:
        print(f"could not find player key for event: {event_type}")
        return None

def get_key_event_opponent(event_type):
    if event_type in ["CHAMPION_KILL"]:
        return "victimId"
    elif event_type in ["WARD_PLACED", "BUILDING_KILL", "CHAMPION_SPECIAL_KILL", "CHAMPION_TRANSFORM", "TURRET_PLATE_DESTROYED", "ELITE_MONSTER_KILL", "WARD_KILL", "ITEM_DESTROYED", "ITEM_UNDO",  "ITEM_PURCHASED", "ITEM_SOLD", "GAME_END", "PAUSE_END", "SKILL_LEVEL_UP", "LEVEL_UP"]:
        return None
    else:
        print(f"could not find opponent key for event: {event_type}")
        return None

def is_event_for_every_player(event_type):
    return event_type in ["PAUSE_END", "GAME_END"]

def get_player_name(participant_map, player_num):
    return participant_map[str(player_num)]['name']

In [ ]:
def create_match_timeline_df(match_timeline):
    datetimes = []
    event_types = []
    event_players = []
    event_opponents = []
    event_assistings = []

    participant_summary = create_match_summoner_dict(match_timeline)

    match_timeline_start = None
    match_timeline_end = None
    for frame in match_timeline['info']['frames']:
        for event in frame['events']:
            dt = None
            event_delta = event['timestamp']
            event_type = event['type']
            event_assisting = None
            event_player = None
            player_key = get_key_event_player(event_type)
            if player_key is not None:
                event_player = get_player_name(participant_summary, event[player_key])

            event_opponent=None
            opponent_key = get_key_event_opponent(event_type)
            if opponent_key is not None:
                event_opponent = get_player_name(participant_summary, event[opponent_key])

            if 'realTimestamp' in event:
                if match_timeline_start is None:
                    match_timeline_start = datetimeutil.utc_datetime_from_ts_millis(event['realTimestamp'])
                    dt = match_timeline_start
                else:
                    match_timeline_end = datetimeutil.utc_datetime_from_ts_millis(event['realTimestamp'])
                    dt = match_timeline_end
            else:
                dt = calc_datetime(match_timeline_start, event_delta)

            if 'assistingParticipantIds' in event:
                event_assisting = []
                for id in event['assistingParticipantIds']:
                    ass = get_player_name(participant_summary, id)
                    event_assisting.append(ass)

            if is_event_for_every_player(event_type):
                for i in range(1, 11):
                    datetimes.append(dt)
                    event_types.append(event_type)
                    event_players.append(get_player_name(participant_summary, i))
                    event_opponents.append(event_opponent)
                    event_assistings.append(event_assisting)
            else:
                datetimes.append(dt)
                event_types.append(event_type)
                event_players.append(event_player)
                event_opponents.append(event_opponent)
                event_assistings.append(event_assisting)

    match_timeline_df = pd.DataFrame({
        'datetime': datetimes,
        'event_types': event_types,
        'event_summoner': event_players,
        'event_opponents': event_opponents,
        'event_assistings': event_assistings
    })
    match_timeline_df = match_timeline_df.set_index('datetime')
    match_timeline_df['rounded'] = match_timeline_df.index.round('S')
    match_timeline_df.sort_index(inplace=True)
    return match_timeline_df

In [ ]:
df_timeline_pentakill = create_match_timeline_df(match_timeline_pentakill)

In [ ]:
# Test fct
df_timeline_pentakill.loc[~df_timeline_pentakill['event_assistings'].isnull()].head()

In [ ]:
# Test fct
df_timeline_pentakill.loc[df_timeline_pentakill['event_summoner'] == 'Minions'].head()

In [ ]:
df_timeline_pentakill.info()

In [ ]:
df_timeline_lost_min_kills = create_match_timeline_df(match_timeline_lost_min_kills)
df_timeline_win_max_killsprees = create_match_timeline_df(match_timeline_win_max_killsprees)

### Number of Events by player for all three matches

In [ ]:
# Prepare orders by player and by event_type
def get_events_by_player_desc_df(df_timeline, match_id):
    df_match_participants = get_match_participant_df(match_id)

    events_by_player_desc = df_timeline.groupby(['event_summoner'])
        .size()
        .reset_index(name='count')
        .sort_values('count', ascending=False)

    events_by_player_desc = events_by_player_desc.merge(df_match_participants[['summonerName', 'win']], how='inner', left_on = 'event_summoner', right_on = 'summonerName')
    events_by_player_desc.drop(columns=['summonerName'], inplace=True)
    return events_by_player_desc

In [ ]:
df_events_by_player_pentakill = get_events_by_player_desc_df(df_timeline_pentakill, match_id_pentakill)
df_events_by_player_lost_min_kills = get_events_by_player_desc_df(df_timeline_lost_min_kills, match_id_lost_min_kills)
df_events_by_player_win_max_killsprees = get_events_by_player_desc_df(df_timeline_win_max_killsprees, match_id_win_max_killsprees)

In [ ]:
def plot_event_per_team(df_events_by_player, matchname):
    with sns.plotting_context('talk'):
        f, ax = plt.subplots(1, 2, figsize = (15, 5), gridspec_kw={'width_ratios': [2, 1]})
        sns.barplot(ax=ax[0], y='event_summoner', x='count', data=df_events_by_player.sort_values('win'), hue='win', hue_order=[False,True])
        ax[0].set_ylabel('')
        ax[0].set_xlabel('Anzahl der Events')

        df_events_by_player.groupby(['win'])
            .sum('count')
            .plot.pie(ax=ax[1], y='count', x='win', autopct="%.1f%%", legend=False)
        ax[1].set_ylabel('')

        f.suptitle(f"Events pro Team: Match {matchname}")
        plt.show()


In [ ]:
plot_event_per_team(df_events_by_player_pentakill, "Pentakill")

In [ ]:
plot_event_per_team(df_events_by_player_lost_min_kills, "Lost Min Kills")

In [ ]:
plot_event_per_team(df_events_by_player_win_max_killsprees, "Win max Killing Sprees")

In [ ]:
def plot_event_count_per_player(df_events_by_player, matchname):
    with sns.plotting_context('talk'):
        f, ax = plt.subplots(figsize = (15, 5))

        sns.barplot(ax=ax, y='event_summoner', x='count', data=df_events_by_player)
        ax.set_title(f"Events pro Team: Match {matchname}")
        ax.set_ylabel('')
        ax.set_xlabel('Anzahl der Events')
        plt.show()

In [ ]:
plot_event_count_per_player(df_events_by_player_pentakill, "Pentakill")

In [ ]:
plot_event_count_per_player(df_events_by_player_lost_min_kills, "Lost Min Kills")

In [ ]:
plot_event_count_per_player(df_events_by_player_win_max_killsprees, "Win max Killing Sprees")


### Number of Events by event type for all three matches

In [ ]:
def plot_events_by_type(df_timeline, matchname):
    order_of_events_by_event_type_desc = df_timeline.groupby(['event_types'])
        .size()
        .reset_index(name='count')
        .sort_values('count', ascending=False)['event_types'].values

    with sns.plotting_context('talk'):
        f, ax = plt.subplots(figsize = (16,10))
        sns.countplot(y="event_types", hue="event_summoner", order=order_of_events_by_event_type_desc, data=df_timeline, ax=ax)
        ax.set_title(f'Anteil der User pro Event: Match {matchname}')
        ax.set_ylabel('')
        ax.set_xlabel('Anzahl der Events')
        plt.show()

In [ ]:
plot_events_by_type(df_timeline_pentakill, "Pentakill")

In [ ]:
plot_events_by_type(df_timeline_lost_min_kills, "Lost Min Kills")

In [ ]:
plot_events_by_type(df_timeline_win_max_killsprees, "Win Max Killsprees")

Not every match has every event-type. For example the CHAMPION_TRANSFORM event only occured in the min-Match.  
The events of ITEM_PURCHASED and ITEM_DESTROYED as well as the LEVEL_UP events and the WARD_PLACED events occur very often.


<a id='visualizeboth'></a>
## Step 4: Visualize game timeline data with chatmessage histogram | ([Jump to Contents](#content))

We will combine the differnt data from the two different data sources and create a combined plot.

### Prepare Chatmessages for our 3 matches


In [ ]:
def print_match_stats(messages_per_sec, df_timeline, matchname, df_events_by_player):
    print(f"+++ !!! +++ Stats of match {matchname} +++ !!! +++")
    print(f"Events in the match: {len(df_timeline)}")
    print(f"Number of messages total: {messages_per_sec['count_messages'].sum()}")
    print("---")
    print("Number of messages by timecategory:")
    print(messages_per_sec.groupby(['timecategory'])['count_messages'].sum())
    print("---")
    print("Events by summoner:")
    print(df_events_by_player)

In [ ]:
def create_messages_per_sec_df(df_session_chat_messages, matchId):
    messages_per_sec = df_session_chat_messages.loc[(no_chatbot_comm_msg_mask) & (df_session_chat_messages['matchId'] == matchId)].resample("1S").size()
    timecategories_df = df_session_chat_messages['timecategory'].loc[~df_session_chat_messages.index.duplicated(keep = 'first')]
    messages_per_sec = messages_per_sec.to_frame().join(timecategories_df)
    messages_per_sec.columns = ['count_messages', 'timecategory']
    messages_per_sec['timecategory'] = messages_per_sec['timecategory'].fillna(method="ffill")
    return messages_per_sec

In [ ]:
messages_per_sec_pentakill = create_messages_per_sec_df(df_session_chat_messages, match_id_pentakill)
messages_per_sec_lost_min_kills = create_messages_per_sec_df(df_session_chat_messages, match_id_lost_min_kills)
messages_per_sec_win_max_killsprees = create_messages_per_sec_df(df_session_chat_messages, match_id_win_max_killsprees)

### Plot match-events and chat histogram



The following Plots consist of two subplots:
    
- The upper plot shows the events of the match with the type of event on the y-axis and the time when it occured on the x-axis.

- The lower plot shows the histogram of the chat messages - the number of chat messages per second. The light blue parts of the histogram show messages before or after the match and the dark blue part of the histogram shows the count of messages during the match.

In [ ]:
def plot_game_event_text_hist(messages_per_sec, df_timeline, summoner_name, matchname, only_kill = False):
    # prepare active and passive player events
    active_player = df_timeline.loc[df_timeline['event_summoner'] == summoner_name][['event_types', 'rounded']]
    passive_player = df_timeline.loc[df_timeline['event_opponents'] == summoner_name][['rounded']]
    passive_player['event_types'] = df_timeline.loc[df_timeline['event_opponents'] == summoner_name]['event_types'] + '_PASSIVE'

    timeline_player = active_player.append(passive_player)
    timeline_player.set_index('rounded', inplace=True)
    timeline_player = timeline_player.sort_values('rounded')

    if (only_kill):
        timeline_player_filtered = timeline_player[timeline_player["event_types"].str.contains('_KILL') == True]
        plot_title = f'KILL- and Start/End-Events and Chat-Histogram for summoner {summoner_name}: Match {matchname}'
    else:
        timeline_player_filtered = timeline_player
        plot_title = f'Events and Chat-Histogram for summoner {summoner_name}: Match {matchname}'

    f, ax = plt.subplots(2, 1, figsize = (20,7), gridspec_kw={'height_ratios': [5, 1]}, sharex=True)

    # events unique of event timeline
    events_unique = timeline_player_filtered['event_types'].unique()
    events_unique = np.sort(events_unique)[::-1] # reverse

    # plot histogram at bottom
    baraxes = messages_per_sec.plot.bar(ax = ax[1], xticks=[], log=True, legend=False)

    v_counts = messages_per_sec['timecategory'].value_counts()
    colors = np.repeat(['#0DA9FF', '#0000FF', '#0DA9FF'], [v_counts['BEFORE_MATCH'], v_counts['DURING_MATCH'], v_counts['AFTER_MATCH']])
    for i, color in enumerate(colors):
        baraxes.containers[0].patches[i].set_color(color)

    # set X axis labels
    x_ticks = [messages_per_sec.index.get_loc(player_idx) for player_idx in timeline_player_filtered.index]
    x_tick_labels = [str(messages_per_sec.index[idx].time()) for idx in x_ticks]

    ax[1].set_xticks(x_ticks)
    ax[1].set_xticklabels(labels=x_tick_labels, rotation=90)

    # plot events at top
    num_events = len(events_unique)
    offsets = list(range(10, num_events * 10 + 1, 10))
    colors = sns.color_palette("bright", num_events).as_hex()
    positions = []

    for i, col in enumerate(events_unique):
        positions.append([messages_per_sec.index.get_loc(idx) for idx in timeline_player_filtered[timeline_player_filtered['event_types'] == col].index])

    ax[0].eventplot(positions, colors=colors, lineoffsets=offsets, linelengths=10)

    # set y labels of upper plot
    ax[0].set_yticks(offsets)
    ax[0].set_yticklabels(labels=events_unique)

    ax[0].set_facecolor((.95, .95, .95, 0.95))
    ax[1].set_facecolor((.95, .95, .95, 0.95))
    plt.subplots_adjust(wspace=0, hspace=0)

    ax[0].set_title(plot_title)
    plt.show()

### Plots from the Pentakill-Match

#### Game-Stats:

In [ ]:
print_match_stats(messages_per_sec_pentakill, df_timeline_pentakill, 'Pentakill', df_events_by_player_pentakill)

#### Plots of Mammothman65 - The pentakiller (3rd most events in match)

In [ ]:
plot_game_event_text_hist(messages_per_sec_pentakill, df_timeline_pentakill, 'MAMMOTHMAN65', 'Pentakill', only_kill=False)

There are a lot of messages in the end of the match. This is some seconds after the Pentakill (5 Champion-Kills, 4 Champion-Special-Kills)

There are also a lot of messages when the player killed the first building and the first champion of the opponents.


In [ ]:
plot_game_event_text_hist(messages_per_sec_pentakill, df_timeline_pentakill, 'MAMMOTHMAN65', 'Pentakill', only_kill=True)

Same plot as above - but less event-types on the y-axis of the first plot.

#### Plots of Yung Fappy - The player with the most events during the match

In [ ]:
plot_game_event_text_hist(messages_per_sec_pentakill, df_timeline_pentakill, 'Yung Fappy', 'Pentakill', only_kill=True)

Only the last building kill of this player would result in a lot of chat messages. This summoner was not killed in the whole match (no CHAMPION_KILL_PASSIVE)

#### Plots of Kitzuo - The player with the second most events during the match

In [ ]:
plot_game_event_text_hist(messages_per_sec_pentakill, df_timeline_pentakill, 'Kitzuo', 'Pentakill', only_kill = True)

At 5:46 the Champion Kill could have lead to reactions in the chat.

#### Plots of Wishuwerehere2 - The player with the least events during the match

In [ ]:
plot_game_event_text_hist(messages_per_sec_pentakill, df_timeline_pentakill, 'Wishuwerehere2', 'Pentakill', only_kill = True)